![Parked car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries, insurance is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance has requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to use simple Logistic Regression, identifying the single feature that results in the best-performing model, as measured by accuracy.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table (below) detailing the column names and descriptions below.

## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [42]:
# Import required libraries
library(readr)
library(dplyr)
library(glue)
library(yardstick)

# Start coding!

In [43]:
# Reading in the CSV file
car_insurance <- read_csv("car_insurance.csv")

# Print the first few rows of the car_insurance
print(head(car_insurance))

Rows: 10000 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (19): id, age, gender, race, driving_experience, education, income, cred...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 6 × 19
      id   age gender  race driving_experience education income credit_score
   <dbl> <dbl>  <dbl> <dbl>              <dbl>     <dbl>  <dbl>        <dbl>
1 569520     3      0     1                  0         2      3        0.629
2 750365     0      1     1                  0         0      0        0.358
3 199901     0      0     1                  0         2      1        0.493
4 478866     0      1     1                  0         3      1        0.206
5 731664     1      1     1                  1         0      1        0.388
6 877557     2      0     1                  2         2      3        0.619
# ℹ 11 more variables: vehicle_ownership <dbl>, vehicle_year <dbl>,
#   married <dbl>, children <dbl>, postal_code <dbl>, annual_mileage <dbl>,
#   vehicle_type <dbl>, speeding_violations <dbl>, duis <dbl>,
#   past_accidents <dbl>, outcome <dbl>


In [44]:
# Print the structure of the car_insurance
print(str(car_insurance))

spc_tbl_ [10,000 × 19] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ id                 : num [1:10000] 569520 750365 199901 478866 731664 ...
 $ age                : num [1:10000] 3 0 0 0 1 2 3 1 2 2 ...
 $ gender             : num [1:10000] 0 1 0 1 1 0 1 0 0 0 ...
 $ race               : num [1:10000] 1 1 1 1 1 1 1 1 1 1 ...
 $ driving_experience : num [1:10000] 0 0 0 0 1 2 3 0 2 0 ...
 $ education          : num [1:10000] 2 0 2 3 0 2 2 3 3 2 ...
 $ income             : num [1:10000] 3 0 1 1 1 3 3 1 1 3 ...
 $ credit_score       : num [1:10000] 0.629 0.358 0.493 0.206 0.388 ...
 $ vehicle_ownership  : num [1:10000] 1 0 1 1 1 1 0 0 0 1 ...
 $ vehicle_year       : num [1:10000] 1 0 0 0 0 1 1 1 0 0 ...
 $ married            : num [1:10000] 0 0 0 0 0 0 1 0 1 0 ...
 $ children           : num [1:10000] 1 0 0 1 0 1 1 1 0 1 ...
 $ postal_code        : num [1:10000] 10238 10238 10238 32765 32765 ...
 $ annual_mileage     : num [1:10000] 12000 16000 11000 11000 12000 13000 13000 14000 13000 1100

In [45]:
# Checking for missing values in each column
print(colSums(is.na(car_insurance)))

                 id                 age              gender                race 
                  0                   0                   0                   0 
 driving_experience           education              income        credit_score 
                  0                   0                   0                 982 
  vehicle_ownership        vehicle_year             married            children 
                  0                   0                   0                   0 
        postal_code      annual_mileage        vehicle_type speeding_violations 
                  0                 957                   0                   0 
               duis      past_accidents             outcome 
                  0                   0                   0 


In [46]:
# Print the summary statistics of the data
print(summary(car_insurance))

       id              age           gender           race       
 Min.   :   101   Min.   :0.00   Min.   :0.000   Min.   :0.0000  
 1st Qu.:249638   1st Qu.:1.00   1st Qu.:0.000   1st Qu.:1.0000  
 Median :501777   Median :1.00   Median :0.000   Median :1.0000  
 Mean   :500522   Mean   :1.49   Mean   :0.499   Mean   :0.9012  
 3rd Qu.:753974   3rd Qu.:2.00   3rd Qu.:1.000   3rd Qu.:1.0000  
 Max.   :999976   Max.   :3.00   Max.   :1.000   Max.   :1.0000  
                                                                 
 driving_experience   education        income     credit_score   
 Min.   :0.000      Min.   :0.00   Min.   :0.0   Min.   :0.0534  
 1st Qu.:0.000      1st Qu.:2.00   1st Qu.:1.0   1st Qu.:0.4172  
 Median :1.000      Median :2.00   Median :2.0   Median :0.5250  
 Mean   :1.069      Mean   :2.01   Mean   :1.9   Mean   :0.5158  
 3rd Qu.:2.000      3rd Qu.:3.00   3rd Qu.:3.0   3rd Qu.:0.6183  
 Max.   :3.000      Max.   :3.00   Max.   :3.0   Max.   :0.9608  
          

In [47]:
# Replace NA values with the mean
car_insurance$credit_score[is.na(car_insurance$credit_score)] <- mean(car_insurance$credit_score, na.rm = TRUE)
car_insurance$annual_mileage[is.na(car_insurance$annual_mileage)] <- mean(car_insurance$annual_mileage, na.rm = TRUE)

colSums(is.na(car_insurance))

id                 age              gender                race 
                  0                   0                   0                   0 
 driving_experience           education              income        credit_score 
                  0                   0                   0                   0 
  vehicle_ownership        vehicle_year             married            children 
                  0                   0                   0                   0 
        postal_code      annual_mileage        vehicle_type speeding_violations 
                  0                   0                   0                   0 
               duis      past_accidents             outcome 
                  0                   0                   0

In [48]:
# Get the names of all columns except for 'id' and 'outcome'
feature_names <- names(car_insurance)[!names(car_insurance) %in% c("id", "outcome")]

# Create a new data frame with these feature columns
features <- subset(car_insurance, select = feature_names)
head(features)

age,gender,race,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,0,1,0,2,3,0.6290273,1,1,0,1,10238,12000,0,0,0,0
0,1,1,0,0,0,0.3577571,0,0,0,0,10238,16000,0,0,0,0
0,0,1,0,2,1,0.4931458,1,0,0,0,10238,11000,0,0,0,0
0,1,1,0,3,1,0.2060129,1,0,0,1,32765,11000,0,0,0,0
1,1,1,1,0,1,0.3883659,1,0,0,0,32765,12000,0,2,0,1
2,0,1,2,2,3,0.6191274,1,1,0,1,10238,13000,0,3,0,3


In [49]:
# Initialize an empty list to store the models and predictions
models <- list()
predictions <- list()

# Loop through each feature
for (feature in feature_names) {
  # Create the formula for the logistic regression model
  formula <- as.formula(paste("outcome ~", feature))
  
  # Fit the logistic regression model
  model <- glm(formula, data = car_insurance, family = binomial())
  
  # Store the model in the list
  models[[feature]] <- model
  
  # Get the predictions from the model
  pred <- round(fitted(model))
  
  # Store the predictions in the list
  predictions[[feature]] <- pred
}

In [50]:
# Initialize an empty list to store the accuracy values
accuracy <- list()

# Loop through each feature
for (feature in feature_names) {
  # Get the predictions for this feature
  pred <- predictions[[feature]]
  
  # Calculate the accuracy
  acc <- length(which(pred == car_insurance$outcome)) / length(car_insurance$outcome)
  
  # Store the accuracy in the list
  accuracy[[feature]] <- acc
}


In [51]:
features_df <- data.frame(features = feature_names)

# Add a new column for accuracy
features_df$accuracy <- NA

# Looping through each feature
for (feature in feature_names) {
  # Getting the accuracy for each features
  acc <- accuracy[[feature]]
  
  # Add the accuracy to the data frame
  features_df$accuracy[features_df$features == feature] <- acc
}

features_df

features,accuracy
<chr>,<dbl>
age,0.7747
gender,0.6867
race,0.6867
driving_experience,0.7771
education,0.6867
income,0.7425
credit_score,0.7054
vehicle_ownership,0.7351
vehicle_year,0.6867


In [52]:
# Find the index of the maximum accuracy
best_index <- which.max(features_df$accuracy)

# Get the best feature and its accuracy
best_feature <- features_df$features[best_index]
best_accuracy <- features_df$accuracy[best_index]


In [53]:
# Create a data frame with the best feature and its accuracy
best_feature_df <- data.frame(
  best_feature = best_feature,
  best_accuracy = best_accuracy
)

best_feature_df

best_feature,best_accuracy
<chr>,<dbl>
driving_experience,0.7771
